# Crypto Currency Exploration
# Data from RapidAPI
## Contact info:
### Sean Pharris - pharris.sean@gmail.com

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crypto-list/digital_currency_list.csv


API used for Crypto data is from RapidAPI - Alpha Vantage. https://rapidapi.com/alphavantage/api/alpha-vantage/   
Documentation - https://www.alphavantage.co/documentation/

In [2]:
# import digital currency list (Contains all Crypto names and codes)
df = pd.read_csv('/kaggle/input/crypto-list/digital_currency_list.csv')

In [3]:
df

,currency code,currency name
0,1ST,FirstBlood
1,2GIVE,GiveCoin
2,808,808Coin
3,AAVE,Aave
4,ABT,ArcBlock
...,...,...
570,ZEN,ZenCash
571,ZET,Zetacoin
572,ZIL,Zilliqa
573,ZLA,Zilla


### Template array for currency CSVs

In [4]:
template = {
    "Time_series":[],
    "Open (USD)":[],
    "High (USD)":[],
    "Low (USD)":[],
    "Close (USD)":[],
    "Volume":[],
    "Market cap (USD)":[],   
}
template

{'Time_series': [],
 'Open (USD)': [],
 'High (USD)': [],
 'Low (USD)': [],
 'Close (USD)': [],
 'Volume': [],
 'Market cap (USD)': []}

### Turn the template into a Data frame to be converted to a CSV

In [5]:
template = pd.DataFrame(template)
template

,Time_series,Open (USD),High (USD),Low (USD),Close (USD),Volume,Market cap (USD)


### Below creates a CSV for all currencies then stores the CSV as a data frame in a dictionary

In [6]:
df_dict = {}
count = 0
while count < len(df['currency name']):
    # each currency's name (+ .csv) is used to create the name of the csv file
    new_csv = df['currency name'][count] + ".csv"
    # the template is used to create the csv file's columns
    template.to_csv(new_csv, index=False)
    # each currency's name (+ _df) is used to create the data frame for each csv
    new_df = df['currency name'][count] + "_df" 
    df_dict[str(new_df)] = new_df
    df_dict[new_df] = pd.read_csv(new_csv)
    count += 1

### The data frame are now accessible through the dictionary.
### Example:

In [7]:
df_dict['Hedera_df']

,Time_series,Open (USD),High (USD),Low (USD),Close (USD),Volume,Market cap (USD)


In [8]:
import requests
import time
import json
from csv import writer

counter = 0

for n, code in enumerate(df['currency code']):
    
    
    
    focused_df = df['currency name'][n]
    
    url = "https://alpha-vantage.p.rapidapi.com/query"
    
    querystring = {"market":"CNY","symbol":code, "function":"DIGITAL_CURRENCY_DAILY"}
    
    headers = {
        'x-rapidapi-host': "alpha-vantage.p.rapidapi.com",
        'x-rapidapi-key': "8e3c6608afmsh5439c071d149e5dp178eeajsn4b6b0d2e820a"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data = response.text

    parse_json = json.loads(data)   
    
    update_csv = focused_df + ".csv"
    update_df = focused_df + "_df"
    
    
    try:
        time_series = parse_json['Time Series (Digital Currency Daily)']
    
        for date in time_series:
            working_time_series = time_series[date]
            working_df = df_dict[update_df]['Time_series']
            if working_df.isin([date]).any(axis=0) == False:
                usd_open = working_time_series['1b. open (USD)']
                usd_high = working_time_series['2b. high (USD)']
                usd_low = working_time_series['3b. low (USD)']
                usd_close = working_time_series['4b. close (USD)']
                vol = working_time_series['5. volume']
                market = working_time_series['6. market cap (USD)']            
    
                list = [date, usd_open, usd_high, usd_low, usd_close, vol, market]
            
                with open(update_csv, 'a', newline='') as f_object:
                    writer_object = writer(f_object)
                    writer_object.writerow(list)
                    f_object.close()
            
            elif working_df.isin(conditional_date).any(axis=0) == True:                       
                pass
    except(KeyError):
        pass
        
    counter += 1
    #print(counter)
    
    csv_path = "./" + update_csv
    df_dict[update_df] = pd.read_csv(csv_path)

In [9]:
df_dict['Cardano_df']

,Time_series,Open (USD),High (USD),Low (USD),Close (USD),Volume,Market cap (USD)


In [10]:
for i in df_dict:
    if len(df_dict[i]) >= 5:
        print(i)

Aave_df
